In [1]:
import os

In [2]:
%pwd

'e:\\Kidney-Disease-Classification-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Kidney-Disease-Classification-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [22]:
from kidneyDiseasePrediction.constants.constant_value import *
from kidneyDiseasePrediction.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [24]:
# Define data ingestion component
import os
import zipfile
import gdown
from kidneyDiseasePrediction import logger
from kidneyDiseasePrediction.utils.common import get_size

In [25]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self) -> str:
        '''
            Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading file from {dataset_url} to {zip_download_dir}")
            
            drive_unique_file_id = dataset_url.split("/")[-2]
            
            download_prefix_url = "https://drive.google.com/uc?export=download&id="
            gdown.download(download_prefix_url + drive_unique_file_id, zip_download_dir)
            
            logger.info(f"Downloaded file from {dataset_url} to {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_file(self) -> None:
        """
            zip file path: str
            Extracts the zip file into the data directory
            Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [26]:
# Define the Data Ingestion Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-10-05 00:56:18,462 : INFO : common  : yaml file: config\config.yaml loaded successfully]
[2025-10-05 00:56:18,479 : INFO : common  : yaml file: params.yaml loaded successfully]
[2025-10-05 00:56:18,484 : INFO : common  : Created directory at: artifacts]
[2025-10-05 00:56:18,486 : INFO : common  : Created directory at: artifacts/data_ingestion]
[2025-10-05 00:56:18,488 : INFO : 1863740088  : Downloading file from https://drive.google.com/file/d/14t1ssholE612Q2eIrxif90J9kjw6-Hyr/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=14t1ssholE612Q2eIrxif90J9kjw6-Hyr
From (redirected): https://drive.google.com/uc?export=download&id=14t1ssholE612Q2eIrxif90J9kjw6-Hyr&confirm=t&uuid=23f4b6e7-ec13-46bc-96f2-597dbae2a9d4
To: e:\Kidney-Disease-Classification-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 46.6M/46.6M [00:04<00:00, 9.80MB/s]


[2025-10-05 00:56:27,251 : INFO : 1863740088  : Downloaded file from https://drive.google.com/file/d/14t1ssholE612Q2eIrxif90J9kjw6-Hyr/view?usp=sharing to artifacts/data_ingestion/data.zip]
